# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Colab Notebooks"

In [3]:
# %cd /content/gdrive/My Drive/Colab Notebooks

In [4]:
import numpy as np
import os
# from scipy.misc import imread, imresize
import datetime
import os
import keras
from skimage import io
from skimage.transform import rescale, resize
import matplotlib.pyplot as plt
from collections import Counter 
import cv2

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
# rn.seed(30) #disabled for experimenting
from keras import backend as K
import tensorflow as tf
# tf.set_random_seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [6]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
# train_doc=train_doc[:100]
# val_doc = val_doc[:100]

In [7]:
# we start with full train data as 1 batch
batch_size = 30 #experiment with the batch size

# we start with 120*120 images and adjust these later for model building process
frames = 15
y = 120 # final image height to be decided
z = 120 # final image width to be decided
channels = 3

In [8]:
# we distribute the images on the basis of size. This is just done for experiment puroposes later.
# outpaths = [train_path] #val_path
def image_segregation(path='train'):
    final_120 =[]
    final_360=[]
    doc = ''
    if(path=='train'):
        op = 'Project_data/train/'
        doc = train_doc
    else:
        op = 'Project_data/val/'
        doc = val_doc
    for f in doc:
        path = f.split(';')[0]
        imgs = os.listdir(op+path)
        for img in imgs:
            image = io.imread(op+path+'/'+img)
            if(image.shape[0]==360):
                final_120.append(op+path+'/'+img) 

            if(image.shape[0]==120):
                final_360.append(op+path+'/'+img)
    return (final_120, final_360)


In [9]:
# make list of images with corresponding sizes
# final_120, final_360 = image_segregation()

In [10]:
# # view random imagees
# selectedIndexes  = []
# for i in range(3):
#     selectedIndexes.append(rn.randint(0,5520))


# s_im_120 = [final_120[i] for i in selectedIndexes]
# s_im_360 = [final_360[i] for i in selectedIndexes]
# print(s_im_120)
# print(s_im_360)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [11]:
def cropImage(image):
    (image_h, image_w, _) = image.shape
    
    h_start=0
    h_end = image_h
    w_start=0
    w_end = image_w
    if(image_h==120):
        h_start=20
        w_start=10
        w_end = 120
    elif image_h==360:
        h_start=30
        w_start=30
        w_end = 320
    return image[h_start:h_end, w_start:w_end]

In [12]:
def resize_image(image, height=y, width=z):
    return  resize(image, (height, width),anti_aliasing=False)

In [13]:
def normalize_image(image):
    norm_image = image - np.min(image)/np.max(image) - np.min(image)
#     norm_image = image - np.percentile(image,5)/ np.percentile(image,95) - np.percentile(image,5)
    return norm_image

In [14]:
#untility to show processed images
def showImage(array, idx):
    a = array[idx]
    image = io.imread(a)
    plt.subplot(2, 2,1)
    plt.imshow(image)

    cropped = cropImage(image)
    plt.subplot(2,2,2)
    plt.imshow(cropped)

    resized = resize_image(cropped)
    plt.subplot(2, 2,3)
    plt.imshow(resized)

    
    normalized = normalize_image(resized)
    plt.subplot(2, 2,4)
    plt.imshow(normalized)

    plt.show()

In [15]:
# plt.figure(figsize=(300,300))
# showImage(s_im_360, 0)

In [16]:
# plt.figure(figsize=(300,300))
# showImage(s_im_120, 0)

In [17]:

def fetch_aug_batchdata(source_path, folder_list, batch_num, batch_size, t,validation):
    
    # intialize variables to store data read from train data
    batch_data = np.zeros((batch_size,frames,y,z,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    #print(batch_data)
    #print(batch_labels)
    # intialize variables for augumented batch data with affine transformation
    batch_data_aug,batch_label_aug = batch_data,batch_labels
    #print(batch_data_aug)
    #print(batch_label_aug)
    # intialize variables for augmented batch data with horizontal flip
    batch_data_flip,batch_label_flip = batch_data,batch_labels
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, 30,2)] 
    
    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # create a random affine to be used in image transformation for buidling agumented data set
        dx, dy = np.random.randint(-1.7, 1.8, 2)
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        
        #  Iterate over the frames for each folder to read them in
        for idx, item in enumerate(img_idx):             
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)           
            # Cropping non symmetric frames
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes 
            resized_image=resize_image(cropImage(image))
            
            #Normal data
            batch_data[folder,idx,:,:,0] = normalize_image(resized_image[:, : , 0])#normalise and feed in the image
            batch_data[folder,idx,:,:,1] = normalize_image(resized_image[:, : , 1])#normalise and feed in the image
            batch_data[folder,idx,:,:,2] = normalize_image(resized_image[:, : , 2])#normalise and feed in the image
            
            a =resized_image.shape[0]
            b =resized_image.shape[1]
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized_image, M, (a,b)))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized_image,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_label_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        if int(t[folder + (batch_num * batch_size)].strip().split(';')[2])==0:
            batch_label_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
            batch_label_flip[folder, 0] = 1                    
        else:
            batch_label_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    #adding the augumented data in the main data.
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_label_final = np.append(batch_labels, batch_label_aug, axis = 0) 
    batch_label_final = np.append(batch_label_final, batch_label_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_label_final= batch_labels
        
    return batch_data_final,batch_label_final

In [18]:
def generator(source_path, folder_list, batch_size, validation=False,ablation=None):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    if(ablation!=None):
        folder_list=folder_list[:ablation]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield fetch_aug_batchdata(source_path, folder_list, batch, batch_size, t,validation)
            
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield fetch_aug_batchdata(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [19]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [20]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


In [21]:
#write your model here

feature_map=[8, 16, 32, 64] # we will expirment with different number of features for different layers
dense_layer_size = [128,64,5]
classes= 5
input_shape = (frames, y, z, channels)
model = Sequential()
# add multiple convulation layers
model.add(TimeDistributed(Conv2D(feature_map[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

model.add(TimeDistributed(Conv2D(feature_map[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
# model.add(keras.layers.Dropout(0.4))

model.add(TimeDistributed(Conv2D(feature_map[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
# model.add(keras.layers.Dropout(0.4))

model.add(TimeDistributed(Conv2D(feature_map[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model.add(GRU(128, return_sequences=False))
model.add(Dense(classes, activation='softmax')) # using Softmax as last layer

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [22]:
optimiser = optimizers.Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 15, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 15, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 15, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 15, 7, 7, 64)      0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [23]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [24]:
model_name = 'model_init_CNN' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  Project_data/train ; batch size = 30
Epoch 1/10


KeyboardInterrupt: 

In [29]:
nb_frames = 30  # lenght of the video frames

## Conv3D model

In [31]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,y,z,channels)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [33]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 60, 60, 16)   

In [34]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [36]:
model_name = 'model_init_Conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [37]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30
Epoch 1/20
